In [7]:
from langchain_openai import ChatOpenAI
from typing import List
from typing import TypedDict, Annotated, Sequence
import os, operator, json
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory
from langchain.chains import LLMMathChain, LLMChain
from langchain.agents.agent_types import AgentType
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
import time
from dateutil import parser
from datetime import datetime


OPENAI_API_KEY="sk-..."

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [8]:
llm = ChatOpenAI(model="gpt-4-1106-preview")

In [9]:
# Redefining the contract data as a JSON string
contract1 = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "05/31/2025",
  "Contract_amount": "$90,000.00"}"""


contract2 = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "18",
  "Contract_amount": "$90,000.00"}"""


contract3 = """{"Sno": 1,
  "Contract_Number": "C887/36",
  "Contract_Date": "11/1/2023",
  "Term_start_date": "12/1/2023",
  "Term_end_date": "18",
  "Contract_amount": [{"item 1" : "SD Card", "Quantity" : "5", "Amount" : "45000$"}, {"item 2": "Tea Bag", "Quantity": "10", "Amount": "45000$"} ]}"""


In [10]:
def parse_date(date_str, dayfirst=False):
    return parser.parse(date_str, dayfirst=dayfirst)

def getting_yearly_amount(contract):
    """ Function use to calculate the yearly amount to be paid for a contract"""
    try:
        contract = json.loads(contract)
    except json.JSONDecodeError as e:
        print('Failed to decode coefficients:', e)
        return None
    # Extracting required information from the contract
    term_contract_date = parse_date(contract['Contract_Date'])
    term_start_date = parse_date(contract['Term_start_date'])
    if len(contract['Term_end_date']) > 5:
        term_end_date = parse_date(contract['Term_end_date'])
        total_months = (term_end_date.year - term_contract_date.year) * 12 + (term_end_date.month - term_contract_date.month)
    else:
        total_months = int(contract['Term_end_date'])

    if type(contract['Contract_amount']) == list:
        contract_amount = sum([float(item['Amount'].replace('$', '').replace(',', '')) for item in contract['Contract_amount']])
    else: 
        contract_amount = float(contract['Contract_amount'].replace('$', '').replace(',', ''))
    # Calculate monthly amount to be paid
    monthly_payment = contract_amount / total_months

    # Calculate the utilized period in months
    utilized_months = (term_start_date.year - term_contract_date.year) * 12 + (term_start_date.month - term_contract_date.month)

    # Calculate the utilized amount so far
    utilized_amount = utilized_months * monthly_payment

    # remaing amount
    remaining_amount = contract_amount - utilized_amount

    return {'utilized_amount': utilized_amount, 'remaining_amount': remaining_amount}



In [11]:
equation_tool = Tool(name='Yearly Amount Estimation', func=getting_yearly_amount, description="Useful for data provided in the json format and estimating amount assciated for each year.")

tools = [equation_tool]

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3, input_key="input", output_key="output",return_messages=True) 

agent_parameters = {
    'agent':'chat-conversational-react-description',
    #'agent':AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    'tools':tools,
    'llm':llm,
    'verbose':True,
    'max_iterations':4,
    'return_intermediate_steps':True,
    'early_stopping_method':'generate',
  
    'memory':memory
}

agent = initialize_agent(**agent_parameters)


In [12]:
%time
output= agent.invoke({"input":f"get each year amount from the given contract {contract3} "})


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


> Entering new AgentExecutor chain...
```json
{
    "action": "Yearly Amount Estimation",
    "action_input": "{\"Sno\": 1, \"Contract_Number\": \"C887/36\", \"Contract_Date\": \"11/1/2023\", \"Term_start_date\": \"12/1/2023\", \"Term_end_date\": \"18\", \"Contract_amount\": [{\"item 1\" : \"SD Card\", \"Quantity\" : \"5\", \"Amount\" : \"45000$\"}, {\"item 2\": \"Tea Bag\", \"Quantity\": \"10\", \"Amount\": \"45000$\"}]}"
}
```
Observation: {'utilized_amount': 5000.0, 'remaining_amount': 85000.0}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Based on the provided information, the utilized amount from the contract is $5,000, and the remaining amount is $85,000."
}
```

> Finished chain.


In [13]:
result=output['intermediate_steps'][-1][-1]
result

{'utilized_amount': 5000.0, 'remaining_amount': 85000.0}

In [14]:
output= agent.invoke({"input":f"get each year amount from the given contract {contract2} "})

result=output['intermediate_steps'][-1][-1]
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Yearly Amount Estimation",
    "action_input": "{\"Sno\": 1, \"Contract_Number\": \"C887/36\", \"Contract_Date\": \"11/1/2023\", \"Term_start_date\": \"12/1/2023\", \"Term_end_date\": \"18\", \"Contract_amount\": \"$90,000.00\"}"
}
```
Observation: {'utilized_amount': 5000.0, 'remaining_amount': 85000.0}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The contract has a utilized amount of $5,000 and a remaining amount of $85,000."
}
```

> Finished chain.


{'utilized_amount': 5000.0, 'remaining_amount': 85000.0}

In [15]:
output= agent.invoke({"input":f"get each year amount from the given contract {contract1} "})

result=output['intermediate_steps'][-1][-1]
result



> Entering new AgentExecutor chain...
```json
{
    "action": "Yearly Amount Estimation",
    "action_input": "{\"Sno\": 1, \"Contract_Number\": \"C887/36\", \"Contract_Date\": \"11/1/2023\", \"Term_start_date\": \"12/1/2023\", \"Term_end_date\": \"05/31/2025\", \"Contract_amount\": \"$90,000.00\"}"
}
```
Observation: {'utilized_amount': 5000.0, 'remaining_amount': 85000.0}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The utilized amount from the contract is $5,000, and the remaining amount is $85,000."
}
```

> Finished chain.


{'utilized_amount': 5000.0, 'remaining_amount': 85000.0}

### Using LLM Math Tool

In [17]:
problem_chain = LLMMathChain.from_llm(llm=llm)

math_tool = Tool.from_function(name="Calculator",func=problem_chain.run,description="Useful for when you need to answer questions \
about math. This tool is only for math questions and nothing else. Only input\
math expressions.")

agent = initialize_agent(
    tools=[math_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

contract1 = {'Sno': 1, 'Contract_Number': 'C887/36', 'Contract_Date': '11/1/2023', 'Term_start_date': '12/1/2023', 'Term_end_date': '05/31/2024', 'Contract_amount': '$90,000.00'}


input_prompt = f""" You are an AI assistant that help us to make mathematical calculation. From the given contract details, calculate the amount to be paid based on the time spend by the auditor.
Here are the details about the contract {contract1}:
Contract_Date - Begining date of the contract and it is in the date format.
Term_start_date - Auditor already worked until this data. This also in the date format.
Term_end_date - The date when the contract will end. It could be either in the date fromat or simply the total number of months taken by contract to complete from the Contract_Date.
Contract_amount - The total amount of the contract. It could be a single amount or an array of items with the amount associated with each item. If it is in the list, calulate the total amount by summing the amount of each item.

Using following steps to calculate the amount to be paid and the remaing amount. also provide the final result in the json format.

1. Total momnth covered by contract = Month between Contract_Date and Term_end_date. Include only month, not the days from the given dates. If the Term_end_date is a number, this number will simply be the total month covered by the contract.
2. Total month spent by auditor = Month between Term_start_date and Contract_Date.
3. Amount associated per month = Contract Amount / Total month covered by contract.
4. Amount utilized by auditor = Amount utilized per month * Total month spent by auditor.
5. Amount remaining = Contract Amount - Amount utilized by auditor.

provide the final result (Amount utilized by auditor and  Amount remaining) in the json format.

"""

In [18]:
output=agent.invoke({'input':input_prompt})


In [19]:
output

{'input': " You are an AI assistant that help us to make mathematical calculation. From the given contract details, calculate the amount to be paid based on the time spend by the auditor.\nHere are the details about the contract {'Sno': 1, 'Contract_Number': 'C887/36', 'Contract_Date': '11/1/2023', 'Term_start_date': '12/1/2023', 'Term_end_date': '05/31/2024', 'Contract_amount': '$90,000.00'}:\nContract_Date - Begining date of the contract and it is in the date format.\nTerm_start_date - Auditor already worked until this data. This also in the date format.\nTerm_end_date - The date when the contract will end. It could be either in the date fromat or simply the total number of months taken by contract to complete from the Contract_Date.\nContract_amount - The total amount of the contract. It could be a single amount or an array of items with the amount associated with each item. If it is in the list, calulate the total amount by summing the amount of each item.\n\nUsing following steps 